ESQUELETO:

Este es el archivo para subir el modelo a la nube.


CÓMO FUNCIONA:

1. Tomar un modelo ya realizado.

2. Lo sube a la nube.

ASEGÚRENSE DE QUE SE COORDINAN LO NECESARIO PARA QUE LOS NOMBRES COINCIDAN (no es de chill cambiar el estándar xd).

In [4]:

import json

#GETTING MY ID:
id = open('../id_CONFIDENTIAL.json', 'r')
mi = json.load(id)
my_id = mi["my_id"]
#my_id

In [5]:
!python3 -m pip install pip==24.0



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\avril\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [6]:
!pip install azureml-sdk


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
ws.delete()

NameError: name 'ws' is not defined

In [10]:
## STEP 1: Set up the workspace.


from azureml.core import Workspace

ws = Workspace.create(name="workspace_bank",
                      subscription_id = my_id,
                      resource_group = "resource_group3",
                      location = "centralindia")



Cleaning up past default Resource Group Deployments on the subscription to avoid limit of 10
Deleting past Resource Group Deployment with name: DeployResourceGroup-6b8437c1d4
Deploying KeyVault with name workspackeyvault912a0f25.
Deploying StorageAccount with name workspacstoragecd5a8e947.
Deploying AppInsights with name workspacinsights9e23755d.
Deployed AppInsights with name workspacinsights9e23755d. Took 28.59 seconds.
Deploying Workspace with name workspace_bank.
Deployed Workspace with name workspace_bank. Took 43.64 seconds.


In [11]:
from azureml.core.model import Model

mname = "model"
registered_model = Model.register(model_path="../best_model.pkl",
                                  model_name=mname,
                                  workspace=ws)

Registering model model


In [12]:
from azureml.core.model import Model

mname_scaler = "model_scaler"
registered_scaler = Model.register(model_path="../scaler.pkl",
                                  model_name=mname,
                                  workspace=ws)

Registering model model


In [61]:
scorepy = """
import json 
import joblib
import pandas as pd
import traceback
from azureml.core.model import Model

def embbed(d):
    try:
        d.columns = d.columns.str.strip()  # Limpiar nombres de columnas
        required_columns = [
            'Net Income to Total Assets', 'ROA(A) before interest and % after tax',
            'ROA(B) before interest and depreciation after tax',
            'ROA(C) before interest and depreciation before interest',
            'Net worth/Assets', 'Debt ratio %',
            'Persistent EPS in the Last Four Seasons',
            'Retained Earnings to Total Assets',
            'Net profit before tax/Paid-in capital',
            'Per Share Net profit before tax (Yuan ¥)',
            'Current Liability to Assets', 'Working Capital to Total Assets',
            "Net Income to Stockholder's Equity", 'Borrowing dependency',
            'Current Liability to Current Assets', 'Liability to Equity',
            'Net Value Per Share (A)', 'Net Value Per Share (B)',
            'Net Value Per Share (C)', 'Current Liability to Equity'
        ]

        missing_columns = [col for col in required_columns if col not in d.columns]
        if missing_columns:
            raise KeyError(f"Missing columns: {missing_columns}")

        return d[required_columns]

    except Exception as e:
        print(f"[ERROR] Unexpected error in embbed(): {e}")
        traceback.print_exc()
        return None

def init():
    global model
    global scaler

    try:
        print("[INFO] Loading model and scaler...")
        
        model_path = Model.get_model_path('model')
        model = joblib.load(model_path)
        print("[INFO] Model loaded successfully.")

        scaler_path = Model.get_model_path('model_scaler')
        scaler = joblib.load(scaler_path)
        print("[INFO] Scaler loaded successfully.")

    except Exception as e:
        print(f"[ERROR] Failed to load model or scaler: {e}")
        traceback.print_exc()
        model = None
        scaler = None  # Si falla, evitamos que cause errores en `run()`

def run(raw_data):
    global model, scaler  # Asegurar que estamos usando las variables globales

    try:
        if model is None or scaler is None:
            raise RuntimeError("[ERROR] Model or scaler not initialized. Check logs for errors.")

        data_dict = json.loads(raw_data)
        if "data" not in data_dict:
            raise ValueError("[ERROR] Input JSON must contain a 'data' key.")

        data = data_dict["data"]
        if not isinstance(data, list):
            raise ValueError("[ERROR] Expected 'data' to be a list of feature dictionaries.")

        df = pd.DataFrame(data)

        print("[INFO] Received data shape:", df.shape)
        print("[INFO] Data columns:", df.columns.tolist())

        embedded_data = embbed(df)
        if embedded_data is None:
            raise ValueError("[ERROR] Data embedding failed due to missing columns.")

        if scaler is None:
            raise RuntimeError("[ERROR] Scaler was not loaded properly in init().")

        scaled_data = scaler.transform(embedded_data)
        print("[INFO] Scaled data shape:", scaled_data.shape)

        result_finals = model.predict(scaled_data)
        print("[INFO] Model prediction output:", result_finals)

        return json.dumps({"predictions": result_finals.tolist()})

    except json.JSONDecodeError as e:
        error_msg = f"[ERROR] Invalid JSON input: {e}"
    except ValueError as e:
        error_msg = str(e)
    except Exception as e:
        error_msg = f"[ERROR] Unexpected error in run(): {e} {traceback.format_exc()}"

    print(error_msg)
    return json.dumps({"error": error_msg})



"""


In [62]:


# Guardar el script en un archivo
file_score = open("score.py", "w")
file_score.write(scorepy)
file_score.close()


In [63]:
## STEP 2: Inference configuration: the blueprints for the German carpenter about how to build your furniture.

from azureml.core.environment import Environment
virtual_env = Environment("env-4-bank")

from azureml.core.conda_dependencies import CondaDependencies
virtual_env.python.conda_dependencies = CondaDependencies.create(conda_packages=['pandas','scikit-learn', 'joblib']) ## Asegúrense de coordinarse con el departamento de modelos para incluir los paquetes correctos. Si no, BOOM!


from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
inference_config = InferenceConfig(
                                environment=virtual_env,
                                entry_script="score.py",
                                )
aci_config = AciWebservice.deploy_configuration(cpu_cores=0.5, memory_gb=1) ## ASEGÚRENSE DE ASIGNAR SUFICIENTE MADERA PARA SUS MUEBLES.

service = Model.deploy(workspace=ws,
                       name='service',
                       models=[registered_model, registered_scaler],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True,
                       )

C:\Users\avril\AppData\Local\Temp\ipykernel_19856\158122555.py:18: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(workspace=ws,


In [64]:
service.wait_for_deployment()

scoring_uri = service.scoring_uri

scoreuri = json.dumps({"URI": [scoring_uri]})
file = open("uri.json", "w")
file.write(scoreuri)
file.close()

ACI service creation operation finished, operation "Succeeded"


In [65]:
print(service.get_logs())


/bin/bash: /azureml-envs/azureml_2e6fc64cd8d9cea1e78b0385b5d059b8/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_2e6fc64cd8d9cea1e78b0385b5d059b8/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_2e6fc64cd8d9cea1e78b0385b5d059b8/lib/libtinfo.so.6: no version information available (required by /bin/bash)
2025-04-03T06:45:43,496179262+00:00 - rsyslog/run 
bash: /azureml-envs/azureml_2e6fc64cd8d9cea1e78b0385b5d059b8/lib/libtinfo.so.6: no version information available (required by bash)
2025-04-03T06:45:43,503713817+00:00 - gunicorn/run 
2025-04-03T06:45:43,505214144+00:00 | gunicorn/run | 
2025-04-03T06:45:43,516530719+00:00 - nginx/run 
2025-04-03T06:45:43,517438572+00:00 | gunicorn/run | ###############################################
2025-04-03T06:45:43,519573461+00:00 | gunicorn/run | AzureML Container Runtime Information
2025-04-03T06:45:43,522099961+00:00 | gunico

In [60]:
service.delete()


Running
2025-04-03 00:42:45-06:00 Check and wait for operation (5ff51cdf-c292-4abf-863e-5ce4a60b1aac) to finish.
2025-04-03 00:42:51-06:00 Deleting service entity.
Succeeded
